# TikTok imports

In [60]:
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as plt
import time
from tqdm.auto import tqdm

import requests
from datetime import datetime

In [18]:
!pip install ensembledata
from ensembledata.api import EDClient

In [19]:
# initialize the ensembledata client
client = EDClient("M2OK8yIEX5UjTkfA")

# get a free token at https://dashboard.ensembledata.com/register
# free trial of 50 units per day for 7 days, starting 23/11/24
#
# costs:
#  - 1 unit = 200 posts in full_hashtag_search :)
#  - 1 unit = 20 posts in hashtag_search
#  - 1 unit = 20 posts in keyword_search or full_keyword_search :(
#  - 1 unit = 30 comments to a post
#  - 1 unit = 30 replies to a comment
#
# documentation available at https://ensembledata.com/apis/docs#tag/Tiktok
# and https://github.com/EnsembleData/tiktok-scraper
# source code available at
# https://github.com/EnsembleData/ensembledata-python/tree/main/ensembledata/api

# Read posts from TikTok

Hashtag search (200 posts at the cost of 1 unit)

In [20]:
# read posts related to an hashtag
result = client.tiktok.full_hashtag_search(
                hashtag = "hurricaneian", # hashtag name
                days = 900,
                remap_output = True, # False for mobile app, True for web
                max_cursor = 2000) # max number of posts fetched,
                                  # cost is 1 unit per 200 posts

In [21]:
# convert json to dataframe
df = pd.json_normalize(result.data["posts"])

In [ ]:
# extract relevant info
df = df[['itemInfos.id', 'itemInfos.createTime', 'itemInfos.authorId',
         'itemInfos.text', 'itemInfos.video.urls', 'itemInfos.shareCount',
         'itemInfos.playCount', 'itemInfos.commentCount']]

# rename columns
df.columns = ['id', 'createTime', 'authorId', 'text', 'video.urls',
              'shareCount', 'playCount', 'commentCount']

# format time to date in string format
df["createTime"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["createTime"]]

In [33]:
# save
df_hashtag = df
df_hashtag.to_csv('ian__tiktok_hashtag_posts.csv', index=False)

In [41]:
for i in range(len(df_hashtag)):
  if df_hashtag["createTime"][i]<"2022-09-14":
    df_hashtag=df_hashtag.drop(i)
  if df_hashtag["createTime"][i]>"2022-10-28":
    df_hashtag=df_hashtag.drop(i)

In [34]:
display(df_hashtag)

,id,createTime,authorId,text,video.urls,shareCount,playCount,commentCount
0,7148607094466317614,2022-09-29,7102120216240981038,Every golfer’s dream #golf #fyp #HurricaneIan ...,[https://v77.tiktokcdn-eu.com/f9d2617806d42cb4...,11893,3682252,968
1,7149263028448546090,2022-09-30,7024839608509072431,#HuracanIan #news #southcarolina #hurricaneIan...,[https://v77.tiktokcdn-eu.com/2726b5140f7c322e...,22442,5159746,1243
2,7148143599966752043,2022-09-27,6716209457804166149,Ian got nothing on us #hurricaneian #florida #...,[https://v77.tiktokcdn-eu.com/3c161cf31f558f4f...,917,9410627,5482
3,7149856973113003269,2022-10-02,6850511303556383750,🌪️ #fyp #uk #usa #hurricaneian,[https://v77.tiktokcdn-eu.com/2566cd082a406d55...,3404,1822982,3991
4,7149240121274027310,2022-09-30,6999646434947187717,Most parts of Family Kingdom is 2-3 feet under...,[https://v77.tiktokcdn-eu.com/10a10a5a2ef8126e...,52012,3709154,2415
...,...,...,...,...,...,...,...,...
1639,7148101715693178154,2022-09-27,6683042468457677830,We’re starting to see images of how Hurricane ...,[https://api16-normal-c-useast2a.tiktokv.com/a...,892,516422,138
1640,7148783146358836526,2022-09-29,6812010676128285701,Replying to @whispersnw Every penny counts💔We ...,[https://api16-normal-c-useast2a.tiktokv.com/a...,1073,67776,402
1641,7149558212289645867,2022-10-01,12640006,@Emily youre a god send for the heads up 🥰 blu...,[https://v77.tiktokcdn-eu.com/ff2a9fa7d885e7d1...,724,531706,403
1642,7149327050229189934,2022-10-01,6580560968357740549,Streamer WingsofRedemption leaves DOG OUTSIDE ...,[https://v77.tiktokcdn-eu.com/792f4c4645bc6077...,156,132460,194


Keyword search (100 posts at the cost of 5 units) - limited to 180 days!

# Read comments from TikTok

Comments search

In [38]:
df_hashtag = pd.read_csv('C:/Users/alepa/PycharmProjects/pythonProject/Network Science/Project/ian__tiktok_hashtag_posts.csv')

# order by comment count
df_hashtag = df_hashtag.sort_values(by=['commentCount'], ascending=False).reset_index(drop=True)

# delete posts with less than 30 comments
df_hashtag = df_hashtag[df_hashtag['commentCount']>=1000]

# state how many
print(f'{len(df_hashtag)} posts for hashtags with more than 30 comments')


377 posts for hashtags with more than 30 comments


In [39]:
display(df_hashtag)

,id,createTime,authorId,text,video.urls,shareCount,playCount,commentCount
0,7419412494910868778,2024-09-27,7394113582072120363,After Hurricane in USA #flood #flooding #flood...,['https://api16-normal-c-useast2a.tiktokv.com/...,24170,4398048,21047
1,7418762255707180321,2024-09-26,7347883042389984289,Tropical Storm Helene in Florida USA helene #s...,['https://v16m.tiktokcdn-eu.com/248d87a6193e72...,62184,17221563,20462
2,7148581794428816683,2022-09-29,6618333664294895621,#FortMyers #HurricaneIan #SWFL #Boat,['https://v77.tiktokcdn-eu.com/a4eae9712fc3d0b...,85321,15116676,17082
3,7149273125157424389,2022-09-30,6666826928797777925,NaN,['https://v45.tiktokcdn-eu.com/74a9e9cc854c19c...,306,523668,13212
4,7149252179885346090,2022-09-30,6943670325058913285,Took a Helicopter to Sanibel Island today 😞#hu...,['https://v77.tiktokcdn-eu.com/c0350578b16e648...,148561,12620461,12378
...,...,...,...,...,...,...,...,...
372,7149743225291705646,2022-10-02,6985372476063499269,P1 wasn’t there when I was there #fyp #trendin...,['https://v45.tiktokcdn-eu.com/5483302db338d26...,1582,1486668,1014
373,7148596654097124651,2022-09-29,6854959161840305158,#ian #hurricaneian,['https://v45.tiktokcdn-eu.com/b18286029a7aa94...,3226,2246473,1012
374,7148893485322489131,2022-09-29,6710314922751181830,Have you ever been in a hurricane before? ⛈ @f...,['https://v45.tiktokcdn-eu.com/862cf9188ec69e2...,648,2650195,1008
375,7148802438617779502,2022-09-29,6642506333986095109,Jk babe ily 🫶🏻 #lineman #linemen #linewife #hu...,['https://api16-normal-c-useast2a.tiktokv.com/...,1118,519064,1007


In [45]:
df_hashtag['createTime'].unique()

array(['2022-09-29', '2022-09-30', '2022-10-05', '2022-10-07',
       '2022-10-01', '2022-09-28', '2022-10-03', '2022-09-27',
       '2022-10-23', '2022-10-10', '2022-09-26', '2022-10-19',
       '2022-10-02', '2022-10-16', '2022-10-04', '2022-09-25',
       '2022-10-11', '2022-10-06', '2022-09-24', '2022-10-22',
       '2022-10-09', '2022-10-15', '2022-10-21', '2022-10-14',
       '2022-10-20', '2022-10-08'], dtype=object)

In [46]:
len(df_hashtag['createTime'].unique()) # we take the comments of 26 days

26

In [54]:
df_hashtag = df_hashtag.loc[df_hashtag.groupby('createTime')['commentCount'].idxmax()]

In [55]:
# order df_hashtag by date
df_hashtag = df_hashtag.sort_values(by=['createTime'], ascending=True).reset_index(drop=True)

df_hashtag

,id,createTime,authorId,text,video.urls,shareCount,playCount,commentCount
0,7146891037787835694,2022-09-24,6865054517386806277,Florida greeting hurricanes #hurricane #florid...,['https://v58.tiktokcdn-eu.com/video/tos/maliv...,13401,1346460,1785
1,7147128979391139114,2022-09-25,6935805351829685254,#hurricaneian #ian #florida #stateofemergency,['https://v58.tiktokcdn-eu.com/video/tos/maliv...,9047,6085682,3099
2,7147742399043931438,2022-09-26,6980433469270033413,#stormwork #hurricane #hurricaneian #florida #...,['https://v77.tiktokcdn-eu.com/4b110c7783bca72...,13258,2484533,5629
3,7147826662787517742,2022-09-27,6758580322201814022,😂😂 Nah people in Florida take evey Hurricane a...,['https://v77.tiktokcdn-eu.com/793d2a53c458230...,11372,1372869,6242
4,7148507639553576235,2022-09-28,7056593308990440494,RESCUE HELP NEEDED AS SOON AS POSSIBLE!! 3 ad...,['https://v77.tiktokcdn-eu.com/68b77fc093c99f4...,9983,1798567,8863
5,7148581794428816683,2022-09-29,6618333664294895621,#FortMyers #HurricaneIan #SWFL #Boat,['https://v77.tiktokcdn-eu.com/a4eae9712fc3d0b...,85321,15116676,17082
6,7149273125157424389,2022-09-30,6666826928797777925,NaN,['https://v45.tiktokcdn-eu.com/74a9e9cc854c19c...,306,523668,13212
7,7149626618011667758,2022-10-01,6951041408883491845,Fort Myers Beach before and after Hurricane Ia...,['https://v58.tiktokcdn-eu.com/video/tos/maliv...,58122,12862858,9081
8,7149856973113003269,2022-10-02,6850511303556383750,🌪️ #fyp #uk #usa #hurricaneian,['https://v77.tiktokcdn-eu.com/2566cd082a406d5...,3404,1822982,3991
9,7150048060478115118,2022-10-03,6991514324134413318,Inside a flooded house #hurricaneian #rain #st...,['https://v77.tiktokcdn-eu.com/cdd905623846926...,15683,7416347,6834


In [62]:
plt.figure(figsize=(10, 6))
plt.bar(df_hashtag["createTime"], df_hashtag["commentCount"], color='b', alpha=0.7)

# Personalizzare il grafico
plt.title('Andamento del numero di Shares nel tempo', fontsize=16)
plt.xlabel('Data', fontsize=14)
plt.ylabel('Number of Shares', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.xticks(rotation=45)

# Mostrare il grafico
plt.tight_layout()
plt.show()

In [64]:
# set search criteria
comments_to_retrieve = 30 # 30 is a safe number = 1 credit!

comments = list()
for i in range(len(df_hashtag)):
    aweme_id = df_hashtag["id"][i]
    cursor = None
    for _ in range(int(comments_to_retrieve/30)):
        result = client.tiktok.post_comments(
            aweme_id = aweme_id,
            cursor = cursor
        )
        comments.extend(result.data["comments"])
        cursor = result.data.get("nextCursor")
        if cursor is None:
            break

In [65]:
# convert json to dataframe
df = pd.json_normalize(comments)

# extract relevant info
df = df[['aweme_id', 'cid', 'create_time', 'text', 'reply_comment_total']]

# format time to date in string format
df["create_time"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["create_time"]]

# order by number of replies
df = df.sort_values(by=['reply_comment_total'], ascending=False).reset_index(drop=True)

# save
df_comments = df
df_comments.to_csv('ian_tiktok_hashtag_comments.csv', index=False)

In [67]:
df_comments

,aweme_id,cid,create_time,text,reply_comment_total
0,7153080462712245546,7154142281814688558,2022-10-14,UPDATE: Rogue was found today and he has passe...,245
1,7149856973113003269,7149867032770069253,2022-10-02,Schools will remain open,144
2,7150685798008999211,7150761601004077870,2022-10-04,He’s doing an excellent job and Florida is gra...,140
3,7149626618011667758,7149662955381080837,2022-10-01,I think there is some misleading camera work a...,127
4,7147742399043931438,7147797067656987438,2022-09-26,We thank everyone coming into the storm to hel...,96
...,...,...,...,...,...
501,7157124572595244330,7157452381259596587,2022-10-22,"Ok, this is my grandpa now, I just love this f...",0
502,7157124572595244330,7157153632685687598,2022-10-22,That’s a miracle. ♥️,0
503,7146891037787835694,7146976940393382699,2022-09-24,hell yeah 😎,0
504,7157615871140285742,7157722167223698218,2022-10-23,"Yo, yo, What did Ethan say when you called him?",0


Replies search

In [ ]:
# set search criteria
replies_to_retrieve = 30 # 30 is a safe number = 1 credit!

replies = list()
for i in range(4): # change the range to get more!
    aweme_id = df_comments["aweme_id"][i]
    cid = df_comments["cid"][i]
    cursor = None
    for _ in range(int(replies_to_retrieve/30)):
        result = client.tiktok.post_comment_replies(
            aweme_id = aweme_id,
            comment_id = cid,
            cursor = cursor
        )
        replies.extend(result.data["comments"])
        cursor = result.data.get("nextCursor")
        if cursor is None:
            break

In [ ]:
# convert json to dataframe
df = pd.json_normalize(replies)

# extract relevant info
df = df[['aweme_id', 'cid', 'reply_id', 'create_time', 'text']]

# format time to date in string format
df["create_time"] = [str(datetime.fromtimestamp(int(i)).date())
                                                for i in df["create_time"]]

# save
df_replies = df
df_replies.to_csv('ian_tiktok_hashtag_replies.csv', index=False)

In [ ]:
df_replies